# IBM Coursera Capstone
### Assignment #1 - Creating a Capstone Project Notebook
##### Brett Wiens

This notebook will be used for the various components of the Capstone project. 

1. Write some markdown to explain that this notebook will be mainly used for the capstone project.
2. Import the pandas library as pd.
3. Import the Numpy library as np.
4. Print the following the statement: Hello Capstone Project Course!

In [1]:
import pandas as pd  ## Import the pandas library as pd to be used later
import numpy as np   ## Import the numpy library as np to be used later

In [2]:
print("Hello Capstone Project Course!")    ## A twist on a classic, "Hello, World!"

Hello Capstone Project Course!
